In [1]:
import folium
from main import main_train

In [2]:
def get_position():
    """緯度, 経度情報"""
    pos = \
        {'清水寺': (34.994856, 135.785046),
        '二条城': (35.01423, 135.748218),
        '伏見稲荷': (34.96714, 135.772672),
        '金閣寺': (35.03937, 135.729243),
        'ギオンコーナー': (35.001551, 135.775822),
        '嵐山': (35.009449, 135.666773),
        '祇園': (35.003782, 135.777245),
        '八坂神社': (35.003656, 135.778553),
        '京都御所': (35.025414, 135.762125),
        '銀閣寺': (35.027021, 135.798206),
        '錦市場': (35.005008, 135.764902),
        '京都タワー': (34.987531, 135.759324),
        '京都駅': (34.985849, 135.758767),
        '龍安寺': (35.034494, 135.718263),
        '伏見': (34.932416, 135.771056),
        '東寺': (34.980598, 135.747786),
        '高台寺': (35.00051, 135.781218),
        '南禅寺': (35.011414, 135.794484),
        '東福寺': (34.976064, 135.773777),
        '平安神宮': (35.015982, 135.782426),
        '嵐山モンキーパーク': (35.011408, 135.676206),
        '東山': (34.992396, 135.775797),
        '河原町': (35.002111, 135.769279),
        '三十三間堂': (34.987885, 135.771713),
        '下鴨神社': (35.038037, 135.772773)}
    return pos

In [3]:
map = folium.Map(location=(34.985849, 135.758767), zoom_start=13)
# プロット
for p in get_position().values():
    folium.Marker(location=p).add_to(map) 
map

In [4]:
from get_data import get_spots, get_time
pos = get_position()
map = folium.Map(location=(34.985849, 135.758767), zoom_start=13)
d = get_time()
spots = get_spots()
lines = []
for i in range(25):
    for j in range(25):
        if i != j and d[i][j] < 10000:
            lines.append({"from": pos[spots[i]], "to": pos[spots[j]]})
# プロット
for p in pos.values():
    folium.Marker(location=p).add_to(map) 
for line in lines:
    folium.PolyLine(
        locations=[line["from"], line["to"]]
    ).add_to(map)

map

In [5]:
def view(route, init=(34.985849, 135.758767), zoom_start=13):
    """
     route を与えるとその route を図示したものを返す.
     route[i] : i 番目に訪れる観光地の名前
     init : mapを表示したときの中心地点 (標準は京都駅)
     zoom_start : mapを表示した時の初期の大きさ
    """
    pos = get_position()
    map = folium.Map(location=init, zoom_start=zoom_start)
    # プロット
    # for p in pos.values():
    #     folium.Marker(location=p).add_to(map) 
    lines = []
    for i in range(len(route) - 1):
        lines.append({"from": pos[route[i]], "to": pos[route[i + 1]]})

    for line in lines:
        folium.PolyLine(
            locations=[line["from"], line["to"]]
        ).add_to(map)

    return map

In [6]:
unique_results = main_train(num_loop=96, train_loop=10000000)

1h 42m 21s


In [37]:
route = unique_results[12][1]
view(route)